# 🗂️ Multi-Modal Document Studio
**Week 3 Exercise — End-to-End Open-Source Document Analysis**

Automates first-pass review of any document (contract, email, article, policy) using only open-source models — no API keys required.

**Produces:**
- Token structure & chat template preview
- Structured LLM brief auto-adapted to document type
- Named entity extraction (people, orgs, locations)
- Per-clause risk scoring
- Overall tone / sentiment analysis

---
**Hardware:** Set `DEVICE = "mps"` for Apple Silicon (default) or `DEVICE = "cuda"` for NVIDIA GPU.

## Cell 1 — Install Dependencies

In [14]:
# Run once — restart kernel after installation
!uv pip install -q transformers torch accelerate pdfplumber gradio sentencepiece optimum-quanto huggingface_hub

print("✅ All packages installed. Restart the kernel, then run from Cell 2 onwards.")

✅ All packages installed. Restart the kernel, then run from Cell 2 onwards.


## Cell 2 — HuggingFace Authentication

Models are downloaded from the [HuggingFace Hub](https://huggingface.co). You need a free account and an access token.

1. Go to https://huggingface.co/settings/tokens
2. Create a token with **Read** permissions
3. Set it as an environment variable before launching Jupyter: `export HF_TOKEN=hf_...`  
   — or paste it directly into the cell below (avoid committing it to git)

In [15]:
import os
from huggingface_hub import login

hf_token = os.environ.get('HF_TOKEN')
if hf_token and hf_token.startswith("hf_"):
  print("HF key looks good so far")
else:
  print("HF key is not set - please click the key in the left sidebar")
login(hf_token, add_to_git_credential=True)

print("✅ Logged in to HuggingFace Hub.")

HF key looks good so far


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Logged in to HuggingFace Hub.


## Cell 3 — Constants & Device Detection

In [16]:
import torch

# ── Device ─────────────────────────────────────────────────────────────────
if torch.backends.mps.is_available(): # Macbook Silicon
    DEVICE = "mps"
    QUANT_BACKEND = "optimum-quanto"   # MPS path
elif torch.cuda.is_available(): # NVIDIA GPU
    DEVICE = "cuda"
    QUANT_BACKEND = "bitsandbytes"     # CUDA path
else:
    DEVICE = "cpu"
    QUANT_BACKEND = "none"

print(f"Device      : {DEVICE}")
print(f"Quant backend: {QUANT_BACKEND}")

# ── Model IDs ───────────────────────────────────────────────────────────────

NER_MODEL         = "dslim/bert-base-NER"
ZERO_SHOT_MODEL    = "facebook/bart-large-mnli"      # Zero-shot classification
SENTIMENT_MODEL    = "nlptown/bert-base-multilingual-uncased-sentiment"
MODEL          = "meta-llama/Llama-3.2-1B-Instruct" 

# ── Risk labels for zero-shot ───────────────────────────────────────────────
RISK_LABELS = [
    "low risk", "medium risk", "high risk",
    "financial obligation", "liability", "termination clause",
    "data privacy", "intellectual property", "indemnification"
]

print("\n✅ Constants set.")

Device      : mps
Quant backend: optimum-quanto

✅ Constants set.


## Cell 3 — Utility: Text Extraction

In [17]:
import pdfplumber, pathlib

def extract_text(source) -> str:
    """Extract text from a PDF path, TXT path, or raw string."""
    if source is None:
        return ""
    if hasattr(source, "name"):          # Gradio UploadedFile
        source = source.name
    # Raw text strings (multi-line or long) are not valid file paths
    if isinstance(source, str) and ('\n' in source or len(source) > 260):
        return source
    p = pathlib.Path(str(source))
    try:
        if p.exists():
            if p.suffix.lower() == ".pdf":
                with pdfplumber.open(p) as pdf:
                    return "\n".join(page.extract_text() or "" for page in pdf.pages)
            else:
                return p.read_text(errors="ignore")
    except OSError:
        pass
    return str(source)

# Quick smoke-test
sample = "This Agreement is entered into between Acme Corp and John Smith on 1 Jan 2025."
# print(extract_text(sample)[:200])
print("✅ Text extractor ready.")

✅ Text extractor ready.


## Cell 4 — Week 3 / Day 2: NER Pipeline

In [18]:
from transformers import pipeline, AutoTokenizer

print(f"Loading NER model onto {DEVICE} …")

ner_pipeline = pipeline(
    "ner",
    model=NER_MODEL,
    aggregation_strategy="simple",
    device=0 if DEVICE == "cuda" else -1  # pipeline uses int device index
)

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def run_ner(text: str) -> str:
    """Return a formatted string of named entities."""
    if not text.strip():
        return "No text provided."
    chunk_size = 400
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    all_entities = []
    print(chunks)
    for chunk in chunks:
        ids = tokenizer.encode(chunk, truncation=True, max_length=512, add_special_tokens=False)
        safe_chunk = tokenizer.decode(ids)
        all_entities.extend(ner_pipeline(safe_chunk))
    
    if not all_entities:
        return "No named entities found."
    
    header = ["| Entity | Group | Score |", "|--------|-------|-------|"]
    rows = [f"| {e['word']} | {e['entity_group']} | {e['score']:.2f} |" for e in all_entities]
    return "\n".join(header + rows)

# Smoke-test
print(run_ner(sample))
print("\n✅ NER pipeline ready.")

Loading NER model onto mps …


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


['This Agreement is entered into between Acme Corp and John Smith on 1 Jan 2025.']
| Entity | Group | Score |
|--------|-------|-------|
| Agreement | MISC | 0.54 |
| A | ORG | 1.00 |
| ##cme Corp | ORG | 0.94 |
| John Smith | ORG | 0.88 |

✅ NER pipeline ready.


## Cell 5 — Week 3 / Day 2: Sentiment Pipeline

In [19]:
print(f"Loading sentiment model …")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    device=0 if DEVICE == "cuda" else -1
)

def run_sentiment(text: str) -> str:
    """Return overall document sentiment."""
    if not text.strip():
        return "No text provided."
    # Sentiment models also cap at 512 tokens — use first 400 words as proxy
    snippet = " ".join(text.split()[:400])
    result = sentiment_pipeline(snippet, truncation=True, max_length=512)[0]
    
    emoji = "🟢" if result["label"] == "POSITIVE" else "🔴"
    return f"{emoji} {result['label']}  (confidence: {result['score']:.2f})"

# print(run_sentiment(sample))
print("✅ Sentiment pipeline ready.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Loading sentiment model …


Device set to use cpu


✅ Sentiment pipeline ready.


## Cell 6 — Week 3 / Day 2: Zero-Shot Risk Scorer

In [25]:
import re, textwrap

print("Loading zero-shot classification model …")
zsc_pipeline = pipeline(
    "zero-shot-classification",
    model=ZERO_SHOT_MODEL,
    device=0 if DEVICE == "cuda" else -1
)

SUPPORTED_DOC_TYPES = ["legal contract", "email", "privacy policy", "research article", "general document"]

DOC_TYPE = "general document"

def run_risk_scoring(text: str) -> str:
    if not text.strip():
        return "No text provided.", "No text provided."
    
    snippet = text[:512]
    DOC_TYPE = zsc_pipeline(snippet, candidate_labels=SUPPORTED_DOC_TYPES, truncation=True, max_length=512)["labels"][0]
    
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())[:10]
    lines = ["| # | Risk | Label | Score | Clause |", "|---|------|-------|-------|--------|"]
    for i, clause in enumerate(sentences, 1):
        result = zsc_pipeline(clause, candidate_labels=RISK_LABELS, truncation=True, max_length=512)
        top_label = result["labels"][0]
        top_score = result["scores"][0]
        risk_icon = (
            "🔴" if "high" in top_label or top_label in ("liability", "indemnification") else
            "🟡" if "medium" in top_label or top_label in ("termination clause", "financial obligation") else
            "🟢"
        )
        snippet = textwrap.shorten(clause, width=90, placeholder="…")
        lines.append(f"| {i:02d} | {risk_icon} | {top_label} | {top_score:.2f} | {snippet} |")

    
    print(f"Document type: {DOC_TYPE}")
    return "\n".join(lines)

# Smoke-test on a two-sentence doc
test_doc = (
    "The Licensor shall not be liable for any indirect damages arising from the use of this software. "
    "Either party may terminate this agreement with 30 days written notice."
)
print(run_risk_scoring(test_doc))
print("\n✅ Risk scorer ready.")

Loading zero-shot classification model …


Device set to use cpu


Document type: legal contract
| # | Risk | Label | Score | Clause |
|---|------|-------|-------|--------|
| 01 | 🔴 | indemnification | 0.52 | The Licensor shall not be liable for any indirect damages arising from the use of this… |
| 02 | 🟡 | termination clause | 0.84 | Either party may terminate this agreement with 30 days written notice. |

✅ Risk scorer ready.


## Cell 7 — Week 3 / Day 3: Tokenizer + Chat Template Preview

In [21]:
from transformers import AutoTokenizer

print("Loading tokenizer …")
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def run_token_preview(text: str) -> str:
    """Show token count and the chat-template-formatted prompt."""
    if not text.strip():
        return "No text provided."
    
    # Raw tokenization
    tokens = tokenizer.encode(text)
    token_count = len(tokens)
    first_10 = tokens[:10]
    
    # Chat template — the Day 3 "aha moment"
    messages = [
        {"role": "system", "content": "You are a professional document analyst."},
        {"role": "user",   "content": f"Briefly summarise this document:\n\n{' '.join(text.split()[:300])}"}
    ]
    chat_prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    chat_tokens = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True
    )
    
    return (
        f"📊 Token Stats\n"
        f"  Raw token count    : {token_count}\n"
        f"  First 10 token IDs : {first_10}\n"
        f"  Chat prompt tokens : {len(chat_tokens)}\n\n"
        f"📝 Chat-Template Prompt (first 600 chars):\n"
        f"{'-'*60}\n"
        f"{chat_prompt[:600]}"
    )

print(run_token_preview(sample))
print("\n✅ Tokenizer ready.")

Loading tokenizer …
📊 Token Stats
  Raw token count    : 21
  First 10 token IDs : [128000, 2028, 23314, 374, 10862, 1139, 1990, 6515, 2727, 22621]
  Chat prompt tokens : 69

📝 Chat-Template Prompt (first 600 chars):
------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 27 Feb 2026

You are a professional document analyst.<|eot_id|><|start_header_id|>user<|end_header_id|>

Briefly summarise this document:

This Agreement is entered into between Acme Corp and John Smith on 1 Jan 2025.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



✅ Tokenizer ready.


## Cell 8 — Week 3 / Days 4–5: LLM Brief Generator (Streaming)

> **Note:** First run will download ~600 MB for TinyLlama. Subsequent runs use the local cache.

In [22]:
import gc, threading
from transformers import AutoModelForCausalLM, TextIteratorStreamer

print(f"Loading LLM on {DEVICE} …")

# Quantization — MPS path uses optimum-quanto; CUDA path uses bitsandbytes
load_kwargs = dict(device_map="auto" if DEVICE == "cuda" else None)

if QUANT_BACKEND == "bitsandbytes":
    from transformers import BitsAndBytesConfig
    load_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)
elif QUANT_BACKEND == "optimum-quanto":
    from optimum.quanto import quantize, qint8
    # We quantize after loading for MPS
    pass

llm_model = AutoModelForCausalLM.from_pretrained(MODEL, **load_kwargs)

if QUANT_BACKEND == "optimum-quanto":
    quantize(llm_model, weights=qint8)

if DEVICE == "mps":
    llm_model = llm_model.to("mps")

llm_model.eval()
print("✅ LLM loaded.")

# ── Document-type detection ────────────────────────────────────────────────
DOC_TYPES = ["legal contract", "email", "privacy policy", "research article", "general document"]

def detect_doc_type(text: str) -> str:
    snippet = text[:512]  # only need a small sample
    result = zsc_pipeline(snippet, candidate_labels=DOC_TYPES, truncation=True, max_length=512)
    return result["labels"][0]

def build_brief_prompt(text: str) -> list[dict]:
    doc_type = detect_doc_type(text)
    print(f"Document type: {doc_type}")
    SYSTEM_PROMPT = (
        f"You are a professional document analyst. The following is a {doc_type}.\n"
        "Provide a structured brief with:\n"
        "1. One-sentence summary\n"
        "2. Key parties or stakeholders\n"
        "3. Main obligations or key points (up to 5 bullet points)\n"
        "4. Notable risks or red flags\n"
        "5. Recommended next action\n"
        "Be concise. Use bullet points."
    )
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": text[:1500]}  # cap context for speed
    ]

def run_llm_brief(text: str, max_new_tokens: int = 400):
    """Generate a structured LLM brief. Yields accumulated text as each token arrives."""
    if not text.strip():
        yield "No text provided."
        return

    messages = build_brief_prompt(text)
    input_ids = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)
    attention_mask = torch.ones_like(input_ids).to(DEVICE)

    streamer = TextIteratorStreamer(
        tokenizer, skip_prompt=True, skip_special_tokens=True
    )

    gen_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        streamer=streamer,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,
    )

    thread = threading.Thread(target=llm_model.generate, kwargs=gen_kwargs)
    thread.start()

    accumulated = ""
    for token_text in streamer:
        accumulated += token_text
        yield accumulated

    thread.join()

    del input_ids
    gc.collect()
    if DEVICE == "cuda":
        torch.cuda.empty_cache()
    elif DEVICE == "mps":
        torch.mps.empty_cache()

# Smoke-test
print("=== LLM Brief (streaming) ===")
# _ = run_llm_brief(test_doc)

Loading LLM on mps …


/Users/davidinyang-etoh/Projects/ai-projects/llm_engineering/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


✅ LLM loaded.
=== LLM Brief (streaming) ===


## Cell 9 — Full Analysis Pipeline (no UI)

In [23]:
def analyse_document(source, verbose: bool = True) -> dict:
    """
    Run the full 5-stage analysis on any document source.
    source: file path (PDF/TXT) or raw text string.
    Returns a dict with keys: text, token_preview, entities, risk_scores, sentiment, llm_brief
    """
    sep = "=" * 70
    
    text = extract_text(source)
    if not text.strip():
        print("⚠️  No text found in document.")
        return {}
    
    results = {"text": text}
    
    # Stage 1 — Token preview (Day 3)
    print(f"{sep}\n[1/5] TOKENIZER PREVIEW\n{sep}")
    results["token_preview"] = run_token_preview(text)
    if verbose: print(results["token_preview"])
    
    # Stage 2 — Named entities (Day 2)
    print(f"\n{sep}\n[2/5] NAMED ENTITY EXTRACTION\n{sep}")
    results["entities"] = run_ner(text)
    if verbose: print(results["entities"])
    
    # Stage 3 — Risk scoring (Day 2)
    print(f"\n{sep}\n[3/5] PER-CLAUSE RISK SCORING\n{sep}")
    results["risk_scores"] = run_risk_scoring(text)
    if verbose: print(results["risk_scores"])
    
    # Stage 4 — Sentiment (Day 2)
    print(f"\n{sep}\n[4/5] OVERALL SENTIMENT\n{sep}")
    results["sentiment"] = run_sentiment(text)
    if verbose: print(results["sentiment"])
    
    # Stage 5 — LLM brief (Days 4-5)
    print(f"\n{sep}\n[5/5] LLM DOCUMENT BRIEF (streaming)\n{sep}")
    results["llm_brief"] = "".join(run_llm_brief(text))
    
    print(f"\n{sep}\n✅ Analysis complete.\n{sep}")
    return results


# ── Run on a sample contract excerpt ──────────────────────────────────────
CONTRACT_SAMPLE = """
SERVICE AGREEMENT

This Agreement is entered into as of January 1, 2025 between Acme Corporation
("Client") and TechSolutions Ltd ("Service Provider").

1. SERVICES. Service Provider agrees to develop and deliver a custom analytics
dashboard by March 31, 2025. Deliverables are specified in Exhibit A.

2. PAYMENT. Client shall pay $50,000 USD within 30 days of invoice. Late payments
shall incur a penalty of 1.5% per month.

3. TERMINATION. Either party may terminate this Agreement with 30 days written
notice. Client may terminate immediately for material breach.

4. LIMITATION OF LIABILITY. In no event shall either party be liable for indirect,
incidental, or consequential damages.

5. GOVERNING LAW. This Agreement shall be governed by the laws of the State of
California. Any disputes shall be resolved by arbitration in San Francisco.
"""

print("Document analysis ready")

# results = analyse_document(CONTRACT_SAMPLE)

Document analysis ready


## Cell 10 — Gradio UI (Optional)

Launches a single-page web UI with file upload and streaming LLM output.

> Run this cell to start the app. A local URL (e.g. `http://127.0.0.1:7860`) will appear below.

In [27]:
import gradio as gr

def gradio_analyse(file_obj, raw_text: str):
    """Gradio handler — streams results stage-by-stage as each completes."""
    source = file_obj if file_obj is not None else raw_text
    if not source:
        yield "", "Please upload a file or paste text.", "", "", "", ""
        return

    text = extract_text(source)
    yield text, "[1/5] Running token preview…", "", "", "", ""

    tok = run_token_preview(text)
    yield text, tok, "[2/5] Running NER…", "", "", ""

    ents = run_ner(text)
    yield text, tok, ents, "[3/5] Running risk scoring…", "", ""

    risk = run_risk_scoring(text)
    yield text, tok, ents, risk, "[4/5] Running sentiment…", ""

    sent = run_sentiment(text)
    yield text, tok, ents, risk, sent, "[5/5] Generating LLM brief…"

    for partial_brief in run_llm_brief(text):
        yield text, tok, ents, risk, sent, partial_brief

with gr.Blocks(title="Multi-Modal Document Studio") as demo:
    gr.Markdown("# 🗂️ Multi-Modal Document Studio\nUpload a PDF/TXT or paste text below.")
    
    with gr.Row():
        file_input = gr.File(label="Upload PDF or TXT", file_types=[".pdf", ".txt"])
        text_input = gr.Textbox(label="Or paste text here", lines=8, placeholder="Paste document text…")
    
    run_btn = gr.Button("🔍 Analyse Document", variant="primary")
    
    with gr.Tabs():
        with gr.Tab("📊 Token Preview"):  tok_out  = gr.Markdown()
        with gr.Tab("🏷️ Named Entities"): ent_out  = gr.Markdown()
        with gr.Tab("⚠️ Risk Scores"):    risk_out = gr.Markdown()
        with gr.Tab("💬 Final Brief"): 
            sent_out = gr.Markdown(label="😐 Sentiment")     
            llm_out  = gr.Markdown(label="💬 Brief")
        
        
        
    
    run_btn.click(
        gradio_analyse,
        inputs=[file_input, text_input],
        outputs=[text_input, tok_out, ent_out, risk_out, sent_out, llm_out]
    )

demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/davidinyang-etoh/Projects/ai-projects/llm_engineering/.venv/lib/python3.12/site-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/davidinyang-etoh/Projects/ai-projects/llm_engineering/.venv/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/davidinyang-etoh/Projects/ai-projects/llm_engineering/.venv/lib/python3.12/site-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/davidinyang-etoh/Projects/ai-projects/llm_engineering/.venv/lib/python3.12/site-packages/starlette/applications.py", line 113, in __call__
    await self.middle

['EMPLOYMENT AGREEMENT This Employment Agreement ("Agreement") is made and entered into as of August 20, 2025, by and between Prompt Computers IO LLC ("Company"), and David Inyang-Etoh ("Employee"). 1. Position, Duties, and Work Location The Company hereby employs the Employee as a Full Stack Developer . The Employee agrees to devote full working time, attention, and best efforts to the employment and to comply with all lawful instructions, policies, rules, and regulations of the Company, including mandatory attendance at all departmental and company-wide meetings. This role is fully remote, and the Employee agrees to maintain a suitable and secure work environment to fulfill their job responsibilities effectively. 2. Term of Employment Employment shall commence on August 20, 2025. The Employee shall undergo a probationary period of one (1) month, which may be extended at the sole discretion of the Company. During this probationary period, the Company may terminate the Employee’s emplo

---

## Concepts Demonstrated

| Cell | Week 3 Concept | What it shows |
|------|---------------|---------------|
| 4 | Day 2 — `pipeline('ner')` | Extract parties, dates, money |
| 5 | Day 2 — `pipeline('sentiment-analysis')` | Overall document tone |
| 6 | Day 2 — `pipeline('zero-shot-classification')` | Per-clause risk without labelled data |
| 7 | Day 3 — `AutoTokenizer` + `apply_chat_template` | Token IDs & prompt format |
| 8 | Day 4 — `AutoModelForCausalLM` + `TextIteratorStreamer` + quantization | Local LLM + streaming |
| 8 | Day 4 — `gc.collect()` + `empty_cache()` | MPS/CUDA memory management |
| 9 | Day 5 — End-to-end chaining | All components wired together |
| 10 | — | Gradio UI with file upload |